In [8]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import urllib.request
from bs4 import BeautifulSoup

In [40]:
total_reviews = []

browser = webdriver.Chrome(executable_path="C:\Caba2015\workspace\project2\dataset\chromedriver.exe")


# 유튜브 브라우저로 이동
url = "https://www.youtube.com/"
browser.get(url)
time.sleep(5)

# 검색창에 커서클릭
elem = browser.find_element_by_xpath("""//*[@id="search"]""")
# 검색어 입력 후 검색
elem.send_keys("음악 취미")
browser.find_element_by_xpath("""//*[@id="search-icon-legacy"]/yt-icon""").click()
time.sleep(3)
# 필터 클릭
browser.find_element_by_xpath("""//*[@id="container"]/ytd-toggle-button-renderer/a""").click()
time.sleep(3)
# '올해' 필터 클릭
browser.find_element_by_xpath("""/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a/div/yt-formatted-string""").click()
time.sleep(3)

# --------------------------------------------------------------------------------------
# body pane을 자동 scroll해주는 코드
body = browser.find_element_by_tag_name('body')

height = browser.find_element_by_xpath("""/html/body/ytd-app/div/ytd-page-manager/ytd-search""").size['height']
while True:
    for i in range(30):
        body.send_keys(Keys.PAGE_DOWN)
    sizecheck = browser.find_element_by_xpath("""/html/body/ytd-app/div/ytd-page-manager/ytd-search""").size['height']
    if height == sizecheck:
        break
    height = sizecheck


# --------------------------------------------------------------------------------------

# 스크롤이 끝까지 내려간 상태에서, video의 url 긁어오기
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

# video의 url이 저장될 list
linklists = []

for element in soup.find_all("div", class_="text-wrapper style-scope ytd-video-renderer"):
    for link in element.find_all("a"):
        hreflink = link.get("href")
        if hreflink[1:6] == 'watch':
            linklists.append(hreflink)
            
print(linklists)

['/watch?v=4HaUBqVxwmw', '/watch?v=FFnlKUbwDFs', '/watch?v=KLj0dPIiorQ', '/watch?v=gFKaeK6s3L4', '/watch?v=Ja6DGbSPX7s', '/watch?v=97g3Ba9u5PU', '/watch?v=fNlDtrAbF7o', '/watch?v=b1fny9qgZzs', '/watch?v=6zUy3gyJtU8', '/watch?v=pgc4-HGK2os', '/watch?v=V83DS3RP_mI', '/watch?v=PfaI7gYiWlI', '/watch?v=SW1Jp16Xm2k', '/watch?v=20N81Q4wvdo', '/watch?v=i2MtAd6R5rY', '/watch?v=jOvprZzZTXc', '/watch?v=1YMfzeFGPds', '/watch?v=9UnAa2Hez0A', '/watch?v=S0E4JQAry0E', '/watch?v=D4rIXa2eEOM', '/watch?v=qmOp2VY_F7Q', '/watch?v=9EP8HOg7Bag', '/watch?v=rwPhxUMc3hk', '/watch?v=X8vYJa6l0Mc', '/watch?v=OY4-dWIeadg', '/watch?v=AEPPx2GKZ-8', '/watch?v=9QQWGXi7-ZA', '/watch?v=D2-R5iC3q2k', '/watch?v=13CBZao3A-Y', '/watch?v=cg9NWMs2oqc', '/watch?v=0Ked-ADfHK0', '/watch?v=QhfCfjC171o', '/watch?v=KBUQIBAyek8', '/watch?v=sTQYf2qdtI4', '/watch?v=3Tk4DCT_w0k', '/watch?v=ngJE3rdIG38', '/watch?v=2KgjdJw-Ljo', '/watch?v=hbjAbB3A_3o', '/watch?v=JaSbWk1FkbE', '/watch?v=Azc7MpoF7vA', '/watch?v=j40yuRxqpSk', '/watch?v=S7JIv

In [32]:
len(linklists)

469

In [33]:
linklists

['/watch?v=WBIxVanbXSg',
 '/watch?v=nQphGsLCYqI',
 '/watch?v=PdJnaSkVESA',
 '/watch?v=00kknXg68P4',
 '/watch?v=U0cfeww2BPE',
 '/watch?v=aS3_e-DDPMg',
 '/watch?v=QIcXkN03Lis',
 '/watch?v=L9Wh8xFXNf8',
 '/watch?v=mMXC6Xl8xgQ',
 '/watch?v=wxlYZGQSdFI',
 '/watch?v=sUTaghM7nco',
 '/watch?v=fiZcc8Wriu4',
 '/watch?v=a6w1t9Mao00',
 '/watch?v=nZTxQeFV8EM',
 '/watch?v=sUAYMq0bkek',
 '/watch?v=RffMaFuBIB8',
 '/watch?v=DwH2IVarqUg',
 '/watch?v=ayLVo0kuBV4',
 '/watch?v=AT1ToZG-9UY',
 '/watch?v=Fzd7MkBAitw',
 '/watch?v=2JbKXoBYfXo',
 '/watch?v=jSSOYJ51ux0',
 '/watch?v=hkJ_eT0RxB8',
 '/watch?v=ilfEekL5Jjo',
 '/watch?v=rTCBxJbk1Q4',
 '/watch?v=iBNhw4SU1Ms',
 '/watch?v=5N1f7KRd3f8',
 '/watch?v=P_sZ16v2SjU',
 '/watch?v=rLelrXlpad8',
 '/watch?v=WoRQB2FoRsI',
 '/watch?v=XaUskj76Pr0',
 '/watch?v=0nocOMrOZ9A',
 '/watch?v=1pLPcfQT89o',
 '/watch?v=YLCN1Wop60A',
 '/watch?v=UsSo0Vf7sDI',
 '/watch?v=ndtVkzAqn2E',
 '/watch?v=EAUU77feh5w',
 '/watch?v=YL_btcLcu3A',
 '/watch?v=qXWis9NYkTM',
 '/watch?v=8i4LgmugSaU',


In [20]:
/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[30]/div[3]/ytd-message-renderer/yt-formatted-string[1]
/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[31]/div[3]/ytd-message-renderer/yt-formatted-string[1]
/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[31]/div[3]/ytd-message-renderer/yt-formatted-string[1]

결과가 더 이상 없습니다.


In [29]:
endpoint = browser.find_element_by_xpath("""/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer""").text

print(endpoint)

if "결과가 더 이상 없습니다." in endpoint:
    print("잡히네.")
else:
    print("못잡네.")

5:27
미술이 취미면 돈이 얼마나 들까?
조회수 3.1만회
8개월 전
김고흐
아크릴화 #취미미술 #아크릴물감 반갑습니다. 반 고흐 그리는 한량 김고흐 입니다. 사실 제가 단순 취미 이상으로 그림을 많이 그리기 ...
자막
16:31
[#모아봤zip] 연예인들의 취미생활 모음📂 송민호 미술 X 이하이 비즈공예 X 윤아 셀프인테리어 X 장우혁 실크스크린 | #온앤오프 #Diggle
조회수 1.6만회
5개월 전
tvN D ENT
모아봤zip #온앤오프 #Diggle #디글 Let's giggle, :Diggle! 방송국놈들이 덕질하는 채널 디글 구독하기! :Diggle ...
5:15
당신은 이 미술 학원에서 5분 안에 도망쳐야 합니다.
조회수 31만회
5개월 전
김앤트
실화를 바탕으로 제작되었지만 실제 인물과는 다른, 미술학원 이야기 입니다. 그림을 잘 그리고 싶은 남다남이 추천 받아서 등록한 ...
자막
10:32
가성비 갑 ! 만원대로 취미미술 🎨 즐기기 ☁️다이소 아크릴 물감 솔직 리뷰 / 토토로 그리기 ☁️ Acrylic painting
조회수 5.1천회
4개월 전
쨩아의미술교실 art
취미미술 #다이소 #아크릴화 00:19 준비물 00:40 스케치 하는 법 02:40 채색 시작 ! 04:20 물감 리뷰 06:58 구름 둥둥 그리기 ...
4K
9:08
미술학원 선택하기 전에 보면 좋은 영상.처음 시작하는 취미 미술.
조회수 2.7천회
5개월 전
arDo 아르도
프리랜서 일러스트레이터 '아르도arDo'의 vlog영상입니다. 일상&그림&취미 영상들을 올리고 있습니다 감사합니다. instagram ...
4K
6:53
수정쌤이 알려주는 취미미술학원 고르는 꿀팁
조회수 85회
1주 전
그리고화실 수정쌤
6:53
[취미미술] 눈 쉽게 그리는 방법 |연필드로잉 |#Easy Drawing for Beginners #with me | |#How to draw eyes |#스케치
조회수 697회
11개월 전
호주작가Plus art
집에서할수있는취미 #그림그리기 #dra

In [ ]:
/html/body/ytd-app/div/ytd-page-manager/ytd-search